In [1]:
library(tidycensus)
library(tidyr)
library(stringr)
library(dplyr)

census_api_key(Sys.getenv("CENSUS_API_KEY"))

# load variable data and transform
variable_data <- load_variables(2020, "acs5", cache = TRUE) %>%
    rename_all(recode,
        name = "variable_key", concept = "dataset",
        label = "variable"
    ) %>%
    mutate(
        dataset = tolower(dataset),
        dataset = gsub(" ", "_", dataset),
        variable = tolower(variable),
        variable = gsub("!!", "_", variable),
        variable = gsub(" ", "_", variable),
        variable = gsub(":", "", variable)
    ) %>%
    select(-geography)


getCensusData <- function(table) {
    # load census data and transform
    census_data <- get_acs(
        geography = "tract",
        table = table,
        year = 2020,
        state = "WA",
        survey = "acs5",
        cache_table = TRUE
    ) %>%
        separate(NAME, c("tract", "county", "state"), sep = ",") %>%
        mutate(
            tract = gsub("Census Tract ", "", tract),
            tract = as.double(tract),
            county = tolower(county),
            county = gsub(" county", "", county)
        ) %>%
        rename_all(recode, variable = "variable_key") %>%
        merge(variable_data, by = "variable_key") %>%
        select(-state, -GEOID, -variable_key)

    return(census_data)
}

getUrbanCensusData <- function(table) {
    # load urban data and transform
    census_data <- get_acs(
    geography = "urban area",
    table = table,
    year = 2020,
    survey = "acs5",
    cache_table = TRUE
    ) %>%
        filter(str_detect(NAME, "Spokane")) %>%
        rename_all(recode, variable = "variable_key") %>%
        merge(variable_data, by = "variable_key") %>%
        select(-GEOID, -NAME, -variable_key)

    return(census_data)
}




Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


To install your API key for use in future sessions, run this function with `install = TRUE`.



In [2]:
head(getCensusData("B05011"))

Getting data from the 2016-2020 5-year ACS

Loading ACS5 variables for 2020 from table B05011 and caching the dataset for faster future access.



,tract,county,estimate,moe,variable,dataset
,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,9501.00,adams,122,52,estimate_total,period_of_naturalization
2,16.02,grays harbor,306,109,estimate_total,period_of_naturalization
3,257.04,king,720,244,estimate_total,period_of_naturalization
4,9613.00,mason,36,41,estimate_total,period_of_naturalization
5,430.00,clark,246,115,estimate_total,period_of_naturalization
6,204.01,king,581,270,estimate_total,period_of_naturalization


In [4]:
getUrbanCensusData("B05011")

Getting data from the 2016-2020 5-year ACS

Loading ACS5 variables for 2020 from table B05011 and caching the dataset for faster future access.



estimate,moe,variable,dataset
<dbl>,<dbl>,<chr>,<chr>
23725,1599,estimate_total,period_of_naturalization
10794,1263,estimate_total_not_a_u.s._citizen,period_of_naturalization
12931,980,estimate_total_naturalized_citizens,period_of_naturalization
2128,456,estimate_total_naturalized_citizens_naturalized_2015_or_later,period_of_naturalization
2407,456,estimate_total_naturalized_citizens_naturalized_2010_to_2014,period_of_naturalization
2613,709,estimate_total_naturalized_citizens_naturalized_2005_to_2009,period_of_naturalization
1280,269,estimate_total_naturalized_citizens_naturalized_2000_to_2004,period_of_naturalization
1102,228,estimate_total_naturalized_citizens_naturalized_1995_to_1999,period_of_naturalization
514,150,estimate_total_naturalized_citizens_naturalized_1990_to_1994,period_of_naturalization


In [4]:
variable_data <- load_variables(2020, "acs5", cache = TRUE) %>%
    rename_all(recode,
        name = "variable_key", concept = "dataset",
        label = "variable"
    ) %>%
    mutate(
        dataset = tolower(dataset),
        dataset = gsub(" ", "_", dataset),
        variable = tolower(variable),
        variable = gsub("!!", "_", variable),
        variable = gsub(" ", "_", variable),
        variable = gsub(":", "", variable)
    ) %>%
    select(-geography)


head(variable_data)

variable_key,variable,dataset
<chr>,<chr>,<chr>
B01001_001,estimate_total,sex_by_age
B01001_002,estimate_total_male,sex_by_age
B01001_003,estimate_total_male_under_5_years,sex_by_age
B01001_004,estimate_total_male_5_to_9_years,sex_by_age
B01001_005,estimate_total_male_10_to_14_years,sex_by_age
B01001_006,estimate_total_male_15_to_17_years,sex_by_age


In [41]:
test <- get_acs(
  geography = "tract",
  table = "B05011",
  year = 2020,
  state = "WA",
  survey = "acs5",
  cache_table = TRUE
) %>%
  separate(NAME, c("tract", "county", "state"), sep = ",") %>%
  select(-state, -GEOID) %>%
  mutate(
    tract = gsub("Census Tract ", "", tract), tract = as.double(tract),
    county = tolower(county), county = gsub(" county", "", county)
  ) %>%
  rename_all(recode, variable = "variable_key")

head(test)

Getting data from the 2016-2020 5-year ACS

Loading ACS5 variables for 2020 from table B05011 and caching the dataset for faster future access.



tract,county,variable_key,estimate,moe
<dbl>,<chr>,<chr>,<dbl>,<dbl>
9501,adams,B05011_001,122,52
9501,adams,B05011_002,28,27
9501,adams,B05011_003,94,43
9501,adams,B05011_004,6,10
9501,adams,B05011_005,0,13
9501,adams,B05011_006,10,10


In [ ]:
getUrbanCensusData <- function(table) {
    variable_data <- load_variables(2020, "acs5", cache = TRUE) %>%
    rename_all(recode,
        name = "variable_key", concept = "dataset",
        label = "variable"
    ) %>%
    mutate(
        dataset = tolower(dataset),
        dataset = gsub(" ", "_", dataset),
        variable = tolower(variable),
        variable = gsub("!!", "_", variable),
        variable = gsub(" ", "_", variable),
        variable = gsub(":", "", variable)
    ) %>%
    select(-geography)

    census_data <- get_acs(
    geography = "urban area",
    table = table,
    year = 2020,
    survey = "acs5",
    cache_table = TRUE
    ) %>%
        filter(str_detect(NAME, "Spokane")) %>%
        rename_all(recode, variable = "variable_key") %>%
        merge(variable_data, by = "variable_key") %>%
        select(-GEOID, -NAME, -variable_key)


    return(census_data)
}

getUrbanCensusData("B05011")

Getting data from the 2016-2020 5-year ACS

Loading ACS5 variables for 2020 from table B05011 and caching the dataset for faster future access.



estimate,moe,variable,dataset
<dbl>,<dbl>,<chr>,<chr>
23725,1599,estimate_total,period_of_naturalization
10794,1263,estimate_total_not_a_u.s._citizen,period_of_naturalization
12931,980,estimate_total_naturalized_citizens,period_of_naturalization
2128,456,estimate_total_naturalized_citizens_naturalized_2015_or_later,period_of_naturalization
2407,456,estimate_total_naturalized_citizens_naturalized_2010_to_2014,period_of_naturalization
2613,709,estimate_total_naturalized_citizens_naturalized_2005_to_2009,period_of_naturalization
1280,269,estimate_total_naturalized_citizens_naturalized_2000_to_2004,period_of_naturalization
1102,228,estimate_total_naturalized_citizens_naturalized_1995_to_1999,period_of_naturalization
514,150,estimate_total_naturalized_citizens_naturalized_1990_to_1994,period_of_naturalization


In [4]:
library(tidycensus)
library(tidyr)
library(stringr)
library(dplyr)

# load variable data and transform
variable_data <- load_variables(2020, "acs5", cache = TRUE) %>%
    rename_all(recode,
        name = "variable_key", concept = "dataset",
        label = "variable"
    ) %>%
    select(-geography)


getCensusData <- function(table) {
    # load census data and transform
    census_data <- get_acs(
        geography = "tract",
        table = table,
        year = 2020,
        state = "WA",
        survey = "acs5",
        cache_table = TRUE
    ) %>%
        separate(NAME, c("tract", "county", "state"), sep = ",") %>%
        rename_all(recode, variable = "variable_key") %>%
        merge(variable_data, by = "variable_key") %>%
        select(-state, -GEOID, -variable_key)

    return(census_data)
}
getCensusData("B05011")

Getting data from the 2016-2020 5-year ACS

Loading ACS5 variables for 2020 from table B05011 and caching the dataset for faster future access.



tract,county,estimate,moe,variable,dataset
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
Census Tract 9501,Adams County,122,52,Estimate!!Total:,PERIOD OF NATURALIZATION
Census Tract 16.02,Grays Harbor County,306,109,Estimate!!Total:,PERIOD OF NATURALIZATION
Census Tract 257.04,King County,720,244,Estimate!!Total:,PERIOD OF NATURALIZATION
Census Tract 9613,Mason County,36,41,Estimate!!Total:,PERIOD OF NATURALIZATION
Census Tract 430,Clark County,246,115,Estimate!!Total:,PERIOD OF NATURALIZATION
Census Tract 204.01,King County,581,270,Estimate!!Total:,PERIOD OF NATURALIZATION
Census Tract 901.01,Kitsap County,298,221,Estimate!!Total:,PERIOD OF NATURALIZATION
Census Tract 402.03,Clark County,158,72,Estimate!!Total:,PERIOD OF NATURALIZATION
Census Tract 53.07,King County,476,174,Estimate!!Total:,PERIOD OF NATURALIZATION
